In [90]:
from urllib.request import Request, urlopen, HTTPError
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import pandas as pd
import csv


keyword = ['데이터 사이언티스트', '데이터 애널리스트', '데이터 엔지니어']
page = 1
driver = webdriver.Chrome()

Title = []
Company = []
Link = []
Location = []
label = []
StackTool = []
Requirement = []
Preference = []

# url, 공고 이름, 기업명, 기업 주소를 찾는 크롤링코드
for search_keyword in keyword:
    driver.get(f'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword={search_keyword}&recruitPage={page}&recruitSort=relation&recruitPageCount=100&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting=')
    ct_result = driver.find_element(By.XPATH, '//*[@id="recruit_info"]/div[1]/span').text
    total_result = re.sub(r'[^0-9]', '', ct_result)
    page_ct = int(int(total_result)/100)

    for pages in range(1, page_ct+2):
        driver.get(f'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword={search_keyword}&recruitPage={pages}&recruitSort=relation&recruitPageCount=100&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting=')
        driver.implicitly_wait(time_to_wait=60)

        div_title = driver.find_elements(By.CLASS_NAME, "job_tit")
        div_company = driver.find_elements(By.CLASS_NAME, "corp_name")
        div_stacktool = driver.find_elements(By.CLASS_NAME, "job_sector")
        div_location = driver.find_elements(By.CLASS_NAME, "job_condition")
        
        try:
            for item in div_title:
                tmp_url = item.find_element(By.TAG_NAME, "a").get_attribute('href')
                Link.append(tmp_url)
                
        except Exception as e:
            Link.append("None")
            print("Link is not exist")
            pass
            
        try:
            for item in div_title:
                tmp_title = item.find_element(By.TAG_NAME, "a").text
                Title.append(tmp_title)
                
        except Exception as e:
            Title.append("None")
            print("Title is not exist")
            pass

        try:
            for item in div_company:
                tmp_cn = item.find_element(By.TAG_NAME, "a").text
                Company.append(tmp_cn)
                label.append(search_keyword)
        except Exception as e:
            Company.append("None")
            print("Company is not exist")
            position_name.append(search_keyword)
            pass
        
        try:
            for item in div_stacktool:
                tmp_stacktool = []
                tmp_stack = item.find_elements(By.TAG_NAME, "a")
                for i in tmp_stack:
                    stack = i.text
                    tmp_stacktool.append(stack)
                StackTool.append(tmp_stacktool)
                
        except Exception as e:
            StackTool.append("None")
            print("StackTool is not exist")
            pass
            

        try:
            for loc in div_location:
                tmp_location = loc.find_element(By.TAG_NAME, "span").text
                Location.append(tmp_location)
                
        except Exception as e:
            Location.append("None")
            print("Location is not exist")
            pass
            
driver.close()

In [115]:
# Create DataFrame
m_df = pd.DataFrame({
    'Title': Title,
    'Company': Company,
    'StackTool' : StackTool,
    'Link': Link,
    'Location': Location,
    'label': label
})

In [121]:
value_counts = m_df.Title.value_counts()
vc = m_df[m_df.Title.isin(value_counts[m_df.Title.value_counts() > 1].index)]
for i in vc.index:
    m_df.drop(i, inplace=True)

In [122]:
from selenium.common.exceptions import NoSuchElementException
import time
from tqdm import tqdm  # Import tqdm

driver = webdriver.Chrome("chromedriver")
Content = []

with tqdm(total=len(m_df['Link'])) as pbar:  # Initialize tqdm progress bar
    for url in m_df['Link']:
        driver.get(url)
        driver.implicitly_wait(time_to_wait=60)
        
        try:
            driver.switch_to.frame("iframe_content_0")
            contents = driver.find_element(By.CLASS_NAME, "user_content").text
            Content.append(contents)
            
        except Exception as e:
            Content.append("None")
            pass
        
        pbar.update(1)  # Update progress bar

driver.close()

 89%|███████████████████████████████████████████████████████████████████▌        | 2256/2540 [2:25:40<18:20,  3.87s/it]


In [ ]:
m_df['Content']=Content

m_df.to_csv("saramin.csv", index=False)